# Modeling

## Phase 3 — Model Training and Evaluation

**Objective:**
Train and evaluate machine learning models using a clean and
leakage-free preprocessing pipeline.

The modeling process follows a progressive strategy:
1. Establish a simple baseline
2. Train more expressive models
3. Compare performance using a consistent metric


In [16]:
import sys
import os

# Add project root to Python path
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline


In [19]:
from src.preprocessing import build_preprocessor
from src.temporal import add_temporal_features

train_df = pd.read_csv("../data/raw/train.csv")
train_df = add_temporal_features(train_df)

X = train_df.drop(columns=["SalePrice"])
y = train_df["SalePrice"]

preprocessor = build_preprocessor()

In [20]:
model = Ridge(alpha=1.0)

pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", model)
])

In [21]:
rmse_scores = -cross_val_score(
    pipeline,
    X,
    y,
    scoring="neg_root_mean_squared_error",
    cv=5
)

rmse_scores.mean(), rmse_scores.std()

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\joaop\OneDrive\Documents\house-price-ml\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\joaop\OneDrive\Documents\house-price-ml\.venv\Lib\site-packages\sklearn\base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\joaop\OneDrive\Documents\house-price-ml\.venv\Lib\site-packages\sklearn\pipeline.py", line 613, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
  File "C:\Users\joaop\OneDrive\Documents\house-p